In [60]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [61]:
data=pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [62]:
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [63]:
label_gender_regression=LabelEncoder()
data['Gender']=label_gender_regression.fit_transform(data['Gender'])
data.head(10)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
5,645,Spain,1,44,8,113755.78,2,1,0,149756.71,1
6,822,France,1,50,7,0.00,2,1,1,10062.80,0
7,376,Germany,0,29,4,115046.74,4,1,0,119346.88,1
8,501,France,1,44,4,142051.07,2,0,1,74940.50,0
9,684,France,1,27,2,134603.88,1,1,1,71725.73,0


In [64]:
##OneHot encoding for geographic column
from sklearn.preprocessing import OneHotEncoder
One_Hot_encoder_regression=OneHotEncoder()
data_transform=One_Hot_encoder_regression.fit_transform(data[['Geography']])
df1=pd.DataFrame(data_transform.toarray(),columns=One_Hot_encoder_regression.get_feature_names_out(['Geography']))
df1.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [65]:
data=pd.concat([data.drop('Geography',axis=1),df1],axis=1)

In [66]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [67]:
with open('label_gender_regression.pkl','wb') as file:
    pickle.dump(label_gender_regression,file)
with open('One_Hot_encoder_regression.pkl','wb') as file:
    pickle.dump(One_Hot_encoder_regression,file)

In [68]:
###Dividing the data into features
X=data.drop('EstimatedSalary',axis=1)
y=data['EstimatedSalary']

##Split data in train and test
X_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

print(X_train.shape)
print(x_test.shape)

#Scaling the data
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
x_test=scaler.transform(x_test)

(8000, 12)
(2000, 12)


In [69]:
with open('scaler_regression.pkl','wb') as file:
    pickle.dump(scaler,file)

In [70]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [71]:
X_train.shape

(8000, 12)

In [72]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)) ,   #Hidden Layer(HL)1 connected to input
    Dense(32,activation='relu'), #HL 2
    Dense(1), #HL 3 Output layer

])
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])


c:\Users\nakul\BankChurn\bankenv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [73]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [74]:
early_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)
log_dir="regressionlogs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensor_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)


In [75]:
hisotry=model.fit(
    X_train,y_train,validation_data=(x_test,y_test),epochs=100,
    callbacks=[tensor_callback,early_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 100396.4766 - mae: 100396.4766 - val_loss: 98588.3750 - val_mae: 98588.3750
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 99885.5156 - mae: 99885.5156 - val_loss: 97538.4453 - val_mae: 97538.4453
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 97980.3594 - mae: 97980.3594 - val_loss: 94668.5156 - val_mae: 94668.5156
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 94022.7109 - mae: 94022.7109 - val_loss: 89644.4062 - val_mae: 89644.4062
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 87991.5703 - mae: 87991.5703 - val_loss: 82776.1484 - val_mae: 82776.1484
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 80407.4844 - mae: 80407.4844 - val_loss: 74915.4297 - val_mae: 74915.4297
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 72282.2656 - mae: 72282.2656 - val_loss: 67265.4297 - val_mae: 67265.4297
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step 

In [76]:
%load_ext tensorboard


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [77]:
%tensorboard --logdir regressionlogs/fit


Reusing TensorBoard on port 6007 (pid 7888), started 0:14:06 ago. (Use '!kill 7888' to kill it.)

In [78]:
test_loss,test_mae=model.evaluate(x_test,y_test)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 50258.9727 - mae: 50258.9727


In [79]:
model.save('regressionmodel.h5')